<a href="https://colab.research.google.com/github/CuriousCaliBoi/AgniKai/blob/main/CoVariance_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║ 1. SETUP  +  QUICK TRAIN                                    ║
# ╚══════════════════════════════════════════════════════════════╝
!pip -q install torch torchvision einops torchinfo \
                 git+https://github.com/yuanli2333/CKA-using-pytorch.git

import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim import Adam

device = "cuda" if torch.cuda.is_available() else "cpu"

# data
train = datasets.MNIST('.', train=True,  download=True,
                       transform=transforms.ToTensor())
test  = datasets.MNIST('.', train=False, download=True,
                       transform=transforms.ToTensor())
train_loader = DataLoader(train, batch_size=512, shuffle=True)
test_loader  = DataLoader(test,  batch_size=1024)

# model
class TinyMLP(nn.Module):
    def __init__(self, use_bn=False):
        super().__init__()
        L = []
        def lin(i,o): return [nn.Linear(i,o), nn.BatchNorm1d(o)] if use_bn else [nn.Linear(i,o)]
        L += [nn.Flatten(), *lin(784,256), nn.ReLU(),
              *lin(256,64) , nn.ReLU(),
              nn.Linear(64,10)]
        self.f = nn.Sequential(*L)
    def forward(self,x): return self.f(x)

net = TinyMLP(use_bn=False).to(device)
opt = Adam(net.parameters(), 1e-3)

# quick train (3 epochs)
for epoch in range(3):
    net.train()
    for x,y in train_loader:
        x,y = x.to(device), y.to(device)
        opt.zero_grad()
        loss = F.cross_entropy(net(x), y)
        loss.backward(); opt.step()
    print(f"epoch {epoch+1}: loss {loss.item():.4f}")